In [1]:
#import the packages
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import ta

import random as rnd


yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [24]:
today = date.today()
ticker1 = 'AAPL' 
start_date = '2022-01-02'
end = today
#df = pdr.get_data_yahoo(ticker1, start=start_date, end=today)
df = yf.download(ticker1, start=start_date, end=today)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,177.830002,182.880005,177.710007,182.009995,179.076584,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,176.803802,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,172.100845,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,169.227921,96904000
2022-01-07,172.889999,174.139999,171.029999,172.169998,169.395172,86709100
...,...,...,...,...,...,...
2024-12-20,248.039993,255.000000,245.690002,254.490005,254.490005,147495300
2024-12-23,254.770004,255.649994,253.449997,255.270004,255.270004,40858800
2024-12-24,255.490005,258.209991,255.289993,258.200012,258.200012,23234700


In [59]:

def rndOption():
    lsPercentage = [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]
    lsPercentageSL = [0.20, 0.3, 0.4]
    sl = rnd.choice(lsPercentageSL)
    tp = rnd.choice(lsPercentage)
    # int between 0 and len df
    index = rnd.randint(125, 250) #len(df)-1
    return sl, tp, index

def getTrade(sl, tp, index):
    trade = {}
    trade['index'] = index
    trade['open'] = df['Open'][index]
    trade['high'] = df['High'][index]
    trade['low'] = df['Low'][index]
    trade['close'] = df['Close'][index]
    trade['slPCT'] = sl
    trade['tpPCT'] = tp
    trade['sl'] = trade['open'] - trade['open'] * sl
    trade['tp'] = trade['open'] + trade['open'] * tp
    return trade

def getTradeResult(trade):
    trade['result'] = 0
    for i in range(trade['index'], len(df)):
        if df['Low'][i] <= trade['sl']:
            trade['result'] = -1
            break
        if df['High'][i] >= trade['tp']:
            trade['result'] = 1
            break
    return trade

#main 
lsTrades = []
for i in range(50):
    sl, tp, index = rndOption()
    trade = getTrade(sl, tp, index)
    trade = getTradeResult(trade)
    lsTrades.append(trade)

# print trades
lsBalance = []
for trade in lsTrades:
    t = trade
    resultTrade = t['sl'] - t['open'] if t['result'] == -1 else t['tp'] - t['open']
    trade['resultTrade'] = resultTrade
    lsBalance.append(resultTrade)
    cad = f'id: {t["index"]}, open: {t["open"]:.2f}, sl: {t["slPCT"]}, tp: {t["tpPCT"]}, {t["sl"]:.2f}, {t["tp"]:.2f}, {t["result"]}, resultTrade: {resultTrade:.2f}'
    print(cad)

print(f'Balance: {sum(lsBalance):.2f}')


id: 227, open: 145.14, sl: 0.2, tp: 0.16, 116.11, 168.36, 1, resultTrade: 23.22
id: 199, open: 145.49, sl: 0.3, tp: 0.04, 101.84, 151.31, 1, resultTrade: 5.82
id: 191, open: 145.81, sl: 0.4, tp: 0.16, 87.49, 169.14, 1, resultTrade: 23.33
id: 131, open: 142.99, sl: 0.3, tp: 0.1, 100.09, 157.29, 1, resultTrade: 14.30
id: 214, open: 140.41, sl: 0.2, tp: 0.2, 112.33, 168.49, 1, resultTrade: 28.08
id: 149, open: 166.37, sl: 0.3, tp: 0.14, 116.46, 189.66, 1, resultTrade: 23.29
id: 204, open: 150.09, sl: 0.3, tp: 0.14, 105.06, 171.10, 1, resultTrade: 21.01
id: 220, open: 149.13, sl: 0.3, tp: 0.12, 104.39, 167.03, 1, resultTrade: 17.90
id: 184, open: 152.74, sl: 0.4, tp: 0.12, 91.64, 171.07, 1, resultTrade: 18.33
id: 196, open: 134.99, sl: 0.2, tp: 0.16, 107.99, 156.59, 1, resultTrade: 21.60
id: 232, open: 147.77, sl: 0.4, tp: 0.06, 88.66, 156.64, 1, resultTrade: 8.87
id: 188, open: 138.21, sl: 0.4, tp: 0.16, 82.93, 160.32, 1, resultTrade: 22.11
id: 155, open: 172.78, sl: 0.3, tp: 0.16, 120.95

In [60]:
# sort trades by resultTrade
lsTrades.sort(key=lambda x: x['resultTrade'], reverse=True)
print('Sorted')
for trade in lsTrades:
    t = trade
    cad = f'id: {t["index"]}, open: {t["open"]:.2f}, sl: {t["slPCT"]}, tp: {t["tpPCT"]}, {t["sl"]:.2f}, {t["tp"]:.2f}, {t["result"]}, resultTrade: {t["resultTrade"]:.2f}'
    print(cad)
    

Sorted
id: 171, open: 154.64, sl: 0.4, tp: 0.2, 92.78, 185.57, 1, resultTrade: 30.93
id: 222, open: 152.31, sl: 0.3, tp: 0.2, 106.62, 182.77, 1, resultTrade: 30.46
id: 191, open: 145.81, sl: 0.4, tp: 0.2, 87.49, 174.97, 1, resultTrade: 29.16
id: 239, open: 145.35, sl: 0.4, tp: 0.2, 87.21, 174.42, 1, resultTrade: 29.07
id: 174, open: 159.90, sl: 0.4, tp: 0.18, 95.94, 188.68, 1, resultTrade: 28.78
id: 214, open: 140.41, sl: 0.2, tp: 0.2, 112.33, 168.49, 1, resultTrade: 28.08
id: 139, open: 154.01, sl: 0.3, tp: 0.18, 107.81, 181.73, 1, resultTrade: 27.72
id: 155, open: 172.78, sl: 0.3, tp: 0.16, 120.95, 200.42, 1, resultTrade: 27.64
id: 191, open: 145.81, sl: 0.3, tp: 0.18, 102.07, 172.06, 1, resultTrade: 26.25
id: 239, open: 145.35, sl: 0.4, tp: 0.18, 87.21, 171.51, 1, resultTrade: 26.16
id: 200, open: 141.69, sl: 0.4, tp: 0.18, 85.01, 167.19, 1, resultTrade: 25.50
id: 139, open: 154.01, sl: 0.3, tp: 0.16, 107.81, 178.65, 1, resultTrade: 24.64
id: 204, open: 150.09, sl: 0.4, tp: 0.16, 90